In [6]:
import cv2
import numpy as np

# Load the image
input_image = "bathymetry.png"
output_image = "lv_depthmap.png"

# Read the image as an RGB image
rgb_image = cv2.imread(input_image, cv2.IMREAD_COLOR)

def rgb_to_grayscale_heatmap(rgb_image):
    # Define the colormap (assuming Jet colormap was used)
    colormap = cv2.COLORMAP_JET

    # Generate the colormap lookup table (LUT)
    num_colors = 256
    colormap_image = np.linspace(0, 255, num_colors, dtype=np.uint8).reshape(-1, 1)
    color_lut = cv2.applyColorMap(colormap_image, colormap).reshape(-1, 3)

    # Convert the LUT to a float representation for smooth transitions
    color_lut = color_lut.astype(np.float32) / 255.0

    # Normalize the input image to float32 [0, 1]
    float_rgb = rgb_image.astype(np.float32) / 255.0
    reshaped_rgb = float_rgb.reshape(-1, 3)

    # Calculate scalar values by interpolating the LUT
    scalar_values = np.zeros(reshaped_rgb.shape[0], dtype=np.float32)
    for i, color in enumerate(reshaped_rgb):
        distances = np.linalg.norm(color_lut - color, axis=1)
        scalar_values[i] = np.argmin(distances)

    # Reshape to original image size
    scalar_image = scalar_values.reshape(rgb_image.shape[:2])

    # Normalize to invert brightness (red -> dark, blue -> light)
    scalar_image = cv2.normalize(scalar_image, None, 255, 0, cv2.NORM_MINMAX)

    # Ensure black and white regions remain unchanged
    grayscale_image = scalar_image.astype(np.uint8)
    black_mask = cv2.inRange(rgb_image, (0, 0, 0), (0, 0, 0))
    white_mask = cv2.inRange(rgb_image, (255, 255, 255), (255, 255, 255))
    grayscale_image[black_mask > 0] = 0
    grayscale_image[white_mask > 0] = 255

    return grayscale_image

# Convert and save the grayscale heatmap
grayscale_image = rgb_to_grayscale_heatmap(rgb_image)
cv2.imwrite(output_image, grayscale_image)


True